In [244]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd #read files
from bs4 import BeautifulSoup
import requests

In [246]:
df = pd.read_csv('/Users/Tico/Documents/Masters/LaMona/ModelTraining/combinedElements.csv')

In [248]:
df

,Unnamed: 0,html,name,price,category
0,0,"<div class=""shopify-section"" id=""shopify-secti...",NaN,NaN,Not Product
1,1,"<div data-section-type=""cart-drawer""><div clas...",NaN,NaN,Not Product
2,2,"<div class=""cart-summary cart-summary--empty c...",NaN,NaN,Not Product
3,3,"<div aria-live=""polite"" class=""cart-summary__i...",NaN,NaN,Not Product
4,4,"<div class=""cart-summary__header cart-summary_...",NaN,NaN,Not Product
...,...,...,...,...,...
2689,2689,"<li class=""grid__item"">\n<link href=""//thecabi...",2 Drawer TV Stand M1,£239.59 GBP,Product
2690,2690,"<li class=""grid__item"">\n<link href=""//thecabi...",Mid Quad Bookcase M1,£0.00 GBP,Product
2691,2691,"<li class=""grid__item"">\n<link href=""//thecabi...",Low Quad Bookcase M1,£181.76 GBP,Product
2692,2692,"<li class=""grid__item"">\n<link href=""//thecabi...",Low 6 Drawer Chest M2,£364.62 GBP,Product


In [250]:
X = df['html']
y = df['category']

In [252]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=22)

In [254]:
#tdidf turns textual data into numerical format
pipeMNB = Pipeline([('tfidf',TfidfVectorizer()),('clf',MultinomialNB())])
pipeCNB = Pipeline([('tfidf',TfidfVectorizer()),('clf',ComplementNB())])
pipeSVC = Pipeline([('tfidf',TfidfVectorizer()),('clf',LinearSVC())])

In [256]:
#training mnb
pipeMNB.fit(X_train, y_train)
predictMNB = pipeMNB.predict(X_test)
print(f"MNB score is: "+str(accuracy_score(y_test,predictMNB)))

MNB score is: 0.9833024118738405


In [258]:
#training cnb
pipeCNB.fit(X_train, y_train)
predictCNB = pipeCNB.predict(X_test)
print(f"CNB score is: "+str(accuracy_score(y_test,predictCNB)))

CNB score is: 0.9907235621521335


In [259]:
#training svc
pipeSVC.fit(X_train, y_train)
predictSVC = pipeSVC.predict(X_test)
print(f"SVC score is: "+str(accuracy_score(y_test,predictSVC)))

/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


SVC score is: 1.0


In [260]:
#check accuracy of models
print(classification_report(y_test,predictSVC))

              precision    recall  f1-score   support

 Not Product       1.00      1.00      1.00       230
     Product       1.00      1.00      1.00       309

    accuracy                           1.00       539
   macro avg       1.00      1.00      1.00       539
weighted avg       1.00      1.00      1.00       539



In [262]:
test = """<li class="grid__item">
<link href="//thecabinetshop.co.uk/cdn/shop/t/18/assets/component-rating.css?v=24573085263941240431657786494" media="all" rel="stylesheet" type="text/css">
<div class="card-wrapper underline-links-hover">
<div class="card card--standard card--media" style="--ratio-percent: 100.0%;">
<div class="card__inner color-background-2 gradient ratio" style="--ratio-percent: 100.0%;"><div class="card__media">
<div class="media media--transparent media--hover-effect">
<img alt="Made to Measure Door" class="motion-reduce" height="1600" sizes="(min-width: 1600px) 367px, (min-width: 990px) calc((100vw - 130px) / 4), (min-width: 750px) calc((100vw - 120px) / 3), calc((100vw - 35px) / 2)" src="//thecabinetshop.co.uk/cdn/shop/products/SHAKERDOORS1.png?v=1659956469&amp;width=533" srcset="//thecabinetshop.co.uk/cdn/shop/products/SHAKERDOORS1.png?v=1659956469&amp;width=165 165w,//thecabinetshop.co.uk/cdn/shop/products/SHAKERDOORS1.png?v=1659956469&amp;width=360 360w,//thecabinetshop.co.uk/cdn/shop/products/SHAKERDOORS1.png?v=1659956469&amp;width=533 533w,//thecabinetshop.co.uk/cdn/shop/products/SHAKERDOORS1.png?v=1659956469&amp;width=720 720w,//thecabinetshop.co.uk/cdn/shop/products/SHAKERDOORS1.png?v=1659956469&amp;width=940 940w,//thecabinetshop.co.uk/cdn/shop/products/SHAKERDOORS1.png?v=1659956469&amp;width=1066 1066w,//thecabinetshop.co.uk/cdn/shop/products/SHAKERDOORS1.png?v=1659956469 1600w" width="1600"/>
</div>
</div><div class="card__content">
<div class="card__information">
<h3 class="card__heading">
<a aria-labelledby="StandardCardNoMediaLink-template--14560278282294__product-grid-4784571809846 NoMediaStandardBadge-template--14560278282294__product-grid-4784571809846" class="full-unstyled-link" href="/products/bespoke-doors" id="StandardCardNoMediaLink-template--14560278282294__product-grid-4784571809846">
                Made to Measure Door
              </a>
</h3>
</div>
<div class="card__badge bottom left"></div>
</div>
</div>
<div class="card__content">
<div class="card__information">
<h3 class="card__heading h5" id="title-template--14560278282294__product-grid-4784571809846">
<a aria-labelledby="CardLink-template--14560278282294__product-grid-4784571809846 Badge-template--14560278282294__product-grid-4784571809846" class="full-unstyled-link" href="/products/bespoke-doors" id="CardLink-template--14560278282294__product-grid-4784571809846">
              Made to Measure Door
            </a>
</h3>
<div class="card-information"><span class="caption-large light"></span>
<div class="price">
<div class="price__container"><div class="price__regular">
<span class="visually-hidden visually-hidden--inline">Regular price</span>
<span class="price-item price-item--regular">
        From ¬£11.52 GBP
      </span>
</div>
<div class="price__sale">
<span class="visually-hidden visually-hidden--inline">Regular price</span>
<span>
<s class="price-item price-item--regular">
</s>
</span><span class="visually-hidden visually-hidden--inline">Sale price</span>
<span class="price-item price-item--sale price-item--last">
        From ¬£11.52 GBP
      </span>
</div>
<small class="unit-price caption hidden">
<span class="visually-hidden">Unit price</span>
<span class="price-item price-item--last">
<span></span>
<span aria-hidden="true">/</span>
<span class="visually-hidden">¬†per¬†</span>
<span>
</span>
</span>
</small>
</div></div>
</div>
</div><div class="card__badge bottom left"></div>
</div>
</div>
</div>
</link></li>
"""

In [266]:
pipeSVC.predict([test])


array(['Product'], dtype=object)

In [268]:
pipeMNB.predict([test])

array(['Product'], dtype='<U11')

In [270]:
pipeCNB.predict([test])

array(['Product'], dtype='<U11')

In [272]:
url = "https://www.yoursclothing.co.uk/collections/crops-shorts"
response = requests.get(url)
soup = BeautifulSoup(response.content,'html.parser')
products = []
divs = soup.find_all('div') 
soup.find('body')


<body class="listing listing--GB">
<h2 class="sr-only">Yours Clothing</h2>
<noscript>
<iframe height="0" src="//www.googletagmanager.com/ns.html?id=GTM-5MMDMT" style="display:none;visibility:hidden" width="0"></iframe>
</noscript>
<input class="d-none" id="no-js-menu" name="no-js-menu" type="checkbox"/>
<header id="main-header">
<div class="esw-warning esw-warning--lg esw-warning--change-country d-none" role="document">
<p aria-label="ESW: Country Selector" class="esw-warning__text text-center py-3 m-0">
            Looks like we have a dedicated website for the country you are shopping on
            <br/>
<button class="esw-warning__button js-trigger-esw text-center text-underline bg-transparent text-white">Click here to choose your country</button>
</p>
</div>
<div class="header-sticky">
<div class="heading-bar">
<div class="container">
<label aria-expanded="false" aria-label="Toggle navigation" class="btn btn-icon btn-nav-toggler js-toggle-menu js-ripple d-lg-none" for="no-js-menu"

In [274]:
products=[]
for child in divs:
    if pipeMNB.predict([str(child)]) == ['Product']:
        products.append(child)

In [275]:
len(products)

1649

In [276]:
products=[]
for child in divs:
    if pipeCNB.predict([str(child)]) == ['Product']:
        products.append(child)
len(products)

1627

In [277]:
products=[]
for child in divs:
    if pipeSVC.predict([str(child)]) == ['Product']:
        products.append(child)
len(products)

502